In [1]:
# Import pandas for data manipulation and analysis
import pandas as pd

# Import regular expressions for string manipulation
import re

# Import Counter from collections for counting occurrences
from collections import Counter

# Import matplotlib.pyplot for data visualization
import matplotlib.pyplot as plt

# Import seaborn for advanced statistical data visualization
import seaborn as sns

In [3]:
# Load the chat file
with open('data/_chat.txt', encoding='utf-8') as file:
    lines = file.readlines()

In [10]:
# Define a pattern to recognize each line of the chat
line_pattern = re.compile(r'\[(\d{2}/\d{2}/\d{4}), (\d{2}:\d{2}:\d{2})] ([^:]+): (.*)')

# Parse the lines
data = []
for line in lines:
    match = line_pattern.match(line)
    if match:
        date = match.group(1)
        time = match.group(2)
        author = match.group(3)
        message = match.group(4)
        data.append([date, time, author, message])

# Create a DataFrame
df = pd.DataFrame(data, columns=['Date', 'Time', 'Author', 'Message'])

# Combine Date and Time into a single datetime column and convert to datetime object
df['Datetime'] = pd.to_datetime(df['Date'] + ' ' + df['Time'], format='%d/%m/%Y %H:%M:%S')
df.drop(['Date', 'Time'], axis=1, inplace=True)  # Drop separate Date and Time columns if not needed

# Display the first few rows to verify
print(df.head())



               Author                                            Message  \
0  RMS DIGITAL REVAMP  ‎Messages and calls are end-to-end encrypted. ...   
1       Carol Wanyama                  ‎Carol Wanyama created this group   
2  RMS DIGITAL REVAMP                           ‎Carol Wanyama added you   
3       Carol Wanyama                           Birthday Cake Loading🥳🥳🥳   
4      Wachira Waruru            Looks great,  when will it be consumed?   

             Datetime  
0 2020-11-18 21:47:16  
1 2020-11-18 21:47:16  
2 2022-10-19 14:00:38  
3 2022-10-19 14:04:26  
4 2022-10-19 14:05:24  
